# Digital People - Whisper Diarization

**NOTA IMPORTANTE**: Prima di eseguire il codice, connettersi a una runtime con GPU andando nel menù in alto "Runtime" -> "Cambia tipo di runtime" e selezionare "T4-GPU".

Una volta connessi alla runtime (è possibile visualizzare lo stato in alto a destra), eseguire i comandi uno alla volta, premendo il pulsante "Play" vicino a ogni cella.

È possibile che nella fase di "*Installazione delle dipendenze necessarie*" alla fine Google Colab chieda di riavviare la runtime. Seguire i passaggi proposti ed eseguire nuovamente il codice da "*Installazione delle dipendenze necessari*e".

In [ ]:
#@title Download del progetto

!git clone https://github.com/luca-martinelli-09/whisper-diarization

In [ ]:
#@title Installazione delle dipendenze necessarie

!pip install openai pyannote.audio pydub pydantic pydantic_settings webvtt-py

In [ ]:
#@title Configurazioni

OPENAI_API_KEY = ""  # @param {type:"string"}
OPENAI_WHISPER_MODEL = "whisper-1"  # @param ["whisper-1", "gpt-4o-transcribe", "gpt-4o-mini-transcribe"]
OPENAI_WHISPER_PROMPT = ""  # @param {type:"string"}

HF_API_KEY = ""  # @param {type:"string"}

PYANNOTE_MODEL = "pyannote/speaker-diarization-community-1"  # @param ["pyannote/speaker-diarization-community-1"]

import os


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_WHISPER_MODEL"] = OPENAI_WHISPER_MODEL
os.environ["OPENAI_WHISPER_PROMPT"] = OPENAI_WHISPER_PROMPT
os.environ["HF_API_KEY"] = HF_API_KEY
os.environ["PYANNOTE_MODEL"] = PYANNOTE_MODEL

In [ ]:
#@title Importazioni

import sys

sys.path.insert(0, "whisper-diarization")

In [ ]:
#@title Configurazioni

import logging
import shutil
from pathlib import Path

from app.diarization import get_pipeline
from app.settings import settings
from app.vtt import create_vtt
from app.whisper import get_transcripts

logger = logging.getLogger(__name__)

settings.tmp_folder.mkdir(exist_ok=True)

In [ ]:
#@title Upload del file

from google.colab import files

uploaded = files.upload()
filename = None

for fn in uploaded.keys():
    filename = fn

filename = Path(filename).absolute()

if not filename.exists():
    logger.error(f"🛑 File {filename} not exists")
    exit(1)

In [ ]:
#@title Avvio della trascrizione

(audio_segment, diarization) = get_pipeline(filename)

transcripts = get_transcripts(diarization, audio_segment)

vtt = create_vtt(transcripts)

In [ ]:
#@title Download del file

export_filename = filename.with_suffix(".vtt")
with open(export_filename, "w", encoding="utf-8") as f:
    vtt.write(f)

shutil.rmtree(settings.tmp_folder.absolute())

files.download(export_filename)